In [72]:
import pandas as pd

text = pd.read_csv('fta_clean.csv')
data = pd.read_csv('trade_clean.csv')
text = text.drop(138)
text = text.set_index('Identifier')

In [73]:
supervised = pd.read_csv('../Data Labelling/machine_labelled.csv', index_col='Identifier')
unsupervised = pd.read_csv('unsupervised.csv', index_col='Identifier')
supervised = supervised.drop(171)
unsupervised = unsupervised.drop(171)

In [74]:
text['Date signed'] = pd.to_datetime(text['Date signed'])

In [75]:
text['Date signed'] = text['Date signed'].dt.year

In [76]:
columns = ['year', 'iso3num_o', 'iso3num_d', 'contig', 'dist', 'comlang_off',
       'comlang_ethno', 'comcol', 'comrelig', 'comleg_pretrans',
       'comleg_posttrans', 'col_dep_ever', 'empire', 'sibling_ever',
       'gdp_ppp_d', 'gdpcap_ppp_o', 'entry_cost_o', 'entry_cost_d',
       'entry_tp_o', 'entry_tp_d', 'tradeflow_baci', 'manuf_tradeflow_baci']

def get_dat(row):
    return data[(data['year'] == row['Date signed']) & (data['iso3_o'] == row['Party 1']) & (data['iso3_d'] == row['Party 2'])].iloc[0][columns]

merged = pd.concat([text,supervised,unsupervised,text.apply(get_dat, axis=1)], axis=1)


In [167]:
merged = merged.drop(['Text', 'Articles'], axis=1)
merged.to_csv('merged.csv')

In [168]:
merged.columns

Index(['Name', 'Type', 'WTO ID', 'Date signed', 'Date into force',
       'Composition', 'Region', 'All WTO', 'Cross Regional', 'Language',
       'Party 1', 'Party 2', 'Total articles', 'Total words',
       'Number of unique words', 'Niceness', 'Class 0', 'Class 1', 'Class 2',
       'year', 'iso3num_o', 'iso3num_d', 'contig', 'dist', 'comlang_off',
       'comlang_ethno', 'comcol', 'comrelig', 'comleg_pretrans',
       'comleg_posttrans', 'col_dep_ever', 'empire', 'sibling_ever',
       'gdp_ppp_d', 'gdpcap_ppp_o', 'entry_cost_o', 'entry_cost_d',
       'entry_tp_o', 'entry_tp_d', 'tradeflow_baci', 'manuf_tradeflow_baci'],
      dtype='object')

In [169]:
# def flatten(x):
#     cols = [f'tradeflow_baci_{year}' for year in x['year']]
#     tradeflows = x.reset_index().drop('index', axis=1)[['tradeflow_baci']].transpose().reset_index().drop('index', axis=1)
#     tradeflows.columns = cols
    
#     manuf_cols = [f'manuf_tradeflow_baci_{year}' for year in x['year']]
#     manuf_tradeflows = x.reset_index().drop('index', axis=1)[['manuf_tradeflow_baci']].transpose().reset_index().drop('index', axis=1)
#     manuf_tradeflows.columns = manuf_cols
    
#     combined = pd.concat([tradeflows, manuf_tradeflows], axis=1)
#     return combined

# data[['year', 'iso3_o', 'iso3_d', 'tradeflow_baci', 'manuf_tradeflow_baci']].groupby(['iso3_o', 'iso3_d']).apply(flatten)
